In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os

In [2]:
import pandas as pd
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
import os, glob
import time

import nltk
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\josed\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def checker2(wrong_options, correct_options):
    i=0
    results = [] 
    for wrong_option in wrong_options:
#        print(wrong_option)
        
#        buenas = wrong_options[wrong_options.Nombre == wrong_option]['Nielsen'].unique()
#        print(buenas)
#        correct_options = correct_options_2[correct_options_2.Desc_Categ.isin(buenas) ]['texto_limpio']
#        print(correct_options)
        
        x = process.extract(wrong_option, correct_options, scorer=fuzz.token_sort_ratio, limit = 1)
#        print(x)
        x = pd.DataFrame(x, columns = ['CONCEPTO_BANORTE','Score2','Familia'])

        #y = process.extract(wrong_option, correct_options, scorer=fuzz.partial_ratio, limit = 6)
        #y = pd.DataFrame(y, columns = ['Similares3','Score3'])

        #x = pd.concat([x, y], axis = 1)
        x = pd.concat([x], axis = 1)

        x['CONCEPTO_BBVA'] = wrong_option
 #       x['tabla_banorte'] = tabla_banorte
 #       x['tabla_santander'] = tabla_santander
        
        results.append(x)
        i+=1
#        print(i)   
    result = pd.concat( results , sort = False)
#    result['articulo_asociado'] = tipo
    return result

In [4]:
def checker3(wrong_options, correct_options):
    i=0
    results = [] 
    for wrong_option in wrong_options:
#        print(wrong_option)
        
#        buenas = wrong_options[wrong_options.Nombre == wrong_option]['Nielsen'].unique()
#        print(buenas)
#        correct_options = correct_options_2[correct_options_2.Desc_Categ.isin(buenas) ]['texto_limpio']
#        print(correct_options)
        
        x = process.extract(wrong_option, correct_options, scorer=fuzz.token_sort_ratio, limit = 1)
#        print(x)
        x = pd.DataFrame(x, columns = ['CONCEPTO_SANTANDER','Score2','Familia'])

        #y = process.extract(wrong_option, correct_options, scorer=fuzz.partial_ratio, limit = 6)
        #y = pd.DataFrame(y, columns = ['Similares3','Score3'])

        #x = pd.concat([x, y], axis = 1)
        x = pd.concat([x], axis = 1)

        x['CONCEPTO_BBVA'] = wrong_option
 #       x['tabla_banorte'] = tabla_banorte
 #       x['tabla_santander'] = tabla_santander
        
        results.append(x)
        i+=1
#        print(i)   
    result = pd.concat( results , sort = False)
#    result['articulo_asociado'] = tipo
    return result

In [7]:
trimestres = ['1T21','2T21','2T20','3T20','4T20']

In [13]:
trimestre = '1T21'
reporte = 'Cuentas_de_Orden'

In [14]:
int(trimestre[-2:])

21

In [15]:
lista_bbva = (os.listdir('C:/Users/josed/Downloads/OUTPUT_BBVA_2021_3/Output/BBVA_'+reporte+'_'+trimestre))      
tablas_bbva = [s for s in lista_bbva if "table" in s]
bbva = pd.DataFrame()
for i in tablas_bbva:
#        print(i)
    globals()[f'bbva_'+ i[-5]] = pd.read_csv('C:/Users/josed/Downloads/OUTPUT_BBVA_2021_3/Output/BBVA_'+reporte+'_'+trimestre+'/'+i)
    globals()[f'bbva_'+ i[-5]].columns = globals()[f'bbva_'+ i[-5]].columns.str.replace(' 20','')
    globals()[f'bbva_'+ i[-5]] = globals()[f'bbva_'+ i[-5]].rename(columns = {globals()[f'bbva_'+ i[-5]].columns[0] : 'CONCEPTO'})
    bbva = pd.concat([bbva,globals()[f'bbva_'+ i[-5]]],axis = 0)


vec_c = []
for columna in bbva.columns:
    nueva_columna = str(columna)+'_BBVA'
    vec_c.append(nueva_columna)
bbva.columns = vec_c 
    
banorte = pd.DataFrame()    
lista_banorte = (os.listdir('C:/Users/josed/Downloads/OUTPUT_BBVA_2021_3/Output/Banorte_'+reporte+'_'+trimestre))      
tablas_banorte = [s for s in lista_banorte if "table" in s]
for i in tablas_banorte:
#        print(i)
    globals()[f'banorte_'+ i[-5]] = pd.read_csv('C:/Users/josed/Downloads/OUTPUT_BBVA_2021_3/Output/Banorte_'+reporte+'_'+trimestre+'/'+i)
    globals()[f'banorte_'+ i[-5]].columns = globals()[f'banorte_'+ i[-5]].columns.str.replace('1T','Mar')
    globals()[f'banorte_'+ i[-5]].columns = globals()[f'banorte_'+ i[-5]].columns.str.replace('2T','Jun')
    globals()[f'banorte_'+ i[-5]].columns = globals()[f'banorte_'+ i[-5]].columns.str.replace('3T','Sep')
    globals()[f'banorte_'+ i[-5]].columns = globals()[f'banorte_'+ i[-5]].columns.str.replace('4T','Dic')
    banorte = pd.concat([banorte,globals()[f'bbva_'+ i[-5]]],axis = 0)
   
vec_c = []
for columna in banorte.columns:
    nueva_columna = str(columna)+'_BANORTE'
    vec_c.append(nueva_columna)
banorte.columns = vec_c

santander = pd.DataFrame()   
lista_santander = (os.listdir('C:/Users/josed/Downloads/OUTPUT_BBVA_2021_3/Output/Santander_'+reporte+'_'+trimestre))      
tablas_santander = [s for s in lista_santander if "table" in s]
anio = int(trimestre[-2:])
for i in tablas_santander:
    globals()[f'santander_'+ i[-5]] = pd.read_csv('C:/Users/josed/Downloads/OUTPUT_BBVA_2021_3/Output/Santander_'+reporte+'_'+trimestre+'/'+i)
    if any("Mar" in s for s in globals()[f'santander_'+ i[-5]].columns):
        print('chido') 
    else:
        globals()[f'santander_'+ i[-5]].columns =globals()[f'santander_'+ i[-5]].values[0]
        
    vec = []
    for columna in globals()[f'santander_1'].values[0]:
        nueva_columna = str(columna)+str(anio)
        if 'Mar '+str(anio) in vec:
            nueva_columna = str(columna)+str(anio-1)
        vec.append(nueva_columna)
        
    globals()[f'santander_'+ i[-5]].columns = vec
    globals()[f'santander_'+ i[-5]].columns = globals()[f'santander_'+ i[-5]].columns.str.replace(' ','')
    santander = pd.concat([santander,globals()[f'bbva_'+ i[-5]]],axis = 0)    
    
vec_c = []
for columna in santander.columns:
    nueva_columna = str(columna)+'_SANTANDER'
    vec_c.append(nueva_columna)
santander.columns = vec_c


In [16]:
bbva

,CONCEPTO_BBVA,"5,141,082 _BBVA",Unnamed: 2_BBVA,565 _BBVA
0,Activos y Pasivos Contingentes,904,NaN,NaN
1,Compromisos Crediticios,"626,480",NaN,NaN
2,Bienes en fideicomiso O mandato,"426,571",NaN,NaN
3,Mandatos,227,NaN,NaN
4,Fideicomisos,"426,344",NaN,NaN
5,Bienes en custodia O en administración,"196,519",NaN,NaN
6,Colaterales recibidos por la entidad,"110,591",NaN,NaN
7,Colaterales recibidos y vendidos O entregados ...,"92,534",NaN,NaN
8,Intereses devengados no cobrados derivados de ...,"3,312",NaN,NaN
9,Otras cuentas de registro,"3,684,171",NaN,NaN


In [115]:
bbva = bbva.drop(columns = 'Unnamed: 6_BBVA')
banorte = banorte.drop(columns = 'Unnamed: 6_BANORTE')
santander = santander.drop(columns = 'Unnamed: 6_SANTANDER')

In [116]:
final = pd.DataFrame()
finals = []

result = checker2(bbva.CONCEPTO_BBVA, banorte.CONCEPTO_BANORTE)


finals.append(result)
final = pd.concat(finals)

final_2 = pd.DataFrame()
finals_2 = []

result_2 = checker3(bbva.CONCEPTO_BBVA, santander.CONCEPTO_SANTANDER)


finals_2.append(result_2)
final_2 = pd.concat(finals_2)

In [117]:
conceptos = final[final.Score2>60]
bbva_2 = bbva.merge(conceptos[['CONCEPTO_BBVA','CONCEPTO_BANORTE']],how = 'left')
balance = bbva_2.merge(banorte , how = 'left')
balance  = balance.drop_duplicates()

In [118]:
balance.columns[7]

'Dic19 _BANORTE'

In [119]:
conceptos = final[final.Score2>60]
bbva_2 = bbva.merge(conceptos[['CONCEPTO_BBVA','CONCEPTO_BANORTE']],how = 'left')
balance = bbva_2.merge(banorte , how = 'left')
balance  = balance.drop_duplicates()


balance[balance.columns[1]] = balance[balance.columns[1]].str.replace(',','')
balance[balance.columns[1]] = balance[balance.columns[1]].str.replace(' ','')
balance[balance.columns[1]] = balance[balance.columns[1]].str.replace('(','')
balance[balance.columns[1]] = balance[balance.columns[1]].str.replace(')','')
balance[balance.columns[1]] = balance[balance.columns[1]].str.replace('-','')
balance[balance.columns[1]] = balance[balance.columns[1]].str.extract('(\d+)').astype(float)


balance[balance.columns[7]] = balance[balance.columns[7]].str.replace(',','')
balance[balance.columns[7]] = balance[balance.columns[7]].str.replace(' ','')
balance[balance.columns[7]] = balance[balance.columns[7]].str.replace('(','')
balance[balance.columns[7]] = balance[balance.columns[7]].str.replace(')','')
balance[balance.columns[7]] = balance[balance.columns[7]].str.replace('-','')
balance[balance.columns[7]] = balance[balance.columns[7]].str.extract('(\d+)').astype(float)


balance['diferencia'] = abs(balance[balance.columns[1]] - balance[balance.columns[7]] )
balance = balance.sort_values('diferencia')
balance = balance.groupby(['CONCEPTO_BBVA']).head(1)

<ipython-input-119-022a56006c0b>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  balance[balance.columns[1]] = balance[balance.columns[1]].str.replace('(','')
<ipython-input-119-022a56006c0b>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  balance[balance.columns[1]] = balance[balance.columns[1]].str.replace(')','')
<ipython-input-119-022a56006c0b>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  balance[balance.columns[7]] = balance[balance.columns[7]].str.replace('(','')
<ipython-input-119-022a56006c0b>:18: FutureWarn

In [120]:
conceptos2 = final_2[final_2.Score2>60]
balance_2 = balance.merge(conceptos2[['CONCEPTO_BBVA','CONCEPTO_SANTANDER']],how = 'left')
balance_2 = balance_2.merge(santander , how = 'left')
balance_2  = balance_2.drop_duplicates()

In [121]:
balance_2.columns[14]

'Dic19 _SANTANDER'

In [122]:
balance_2[balance_2.columns[14]] = balance_2[balance_2.columns[14]].str.replace(',','')
balance_2[balance_2.columns[14]] = balance_2[balance_2.columns[14]].str.replace(' ','')
balance_2[balance_2.columns[14]] = balance_2[balance_2.columns[14]].str.replace('(','')
balance_2[balance_2.columns[14]] = balance_2[balance_2.columns[14]].str.replace(')','')
balance_2[balance_2.columns[14]] = balance_2[balance_2.columns[14]].str.replace('-','')
balance_2[balance_2.columns[14]] = balance_2[balance_2.columns[14]].str.extract('(\d+)').astype(float)


balance_2['diferencia_2'] = abs(balance_2[balance_2.columns[1]] - balance_2[balance_2.columns[14]] )
balance_2 = balance_2.sort_values('diferencia_2')
balance_2 = balance_2.groupby(['CONCEPTO_BBVA']).head(1)

<ipython-input-122-15e867d38a55>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  balance_2[balance_2.columns[14]] = balance_2[balance_2.columns[14]].str.replace('(','')
<ipython-input-122-15e867d38a55>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  balance_2[balance_2.columns[14]] = balance_2[balance_2.columns[14]].str.replace(')','')


In [123]:
balance_2.to_csv('C:/Users/josed/Downloads/BALANCE_GENERAL/balance_general_'+trimestre+'.csv')